In [1]:
import numpy as np

In [2]:
def convolution(x,
                ftr,
                strd):

    '''

    ###args

    x : input of convolution.
            [BATCH, HEIGHT, WIDTH, CHANNEL]
    ftr : filter for convolution operation.
            [IN CHANNEL, FILTER H, FILTER W, OUT CHANNEL]
    strd : filter stride number. 
            [H STRIDE, W STRIDE]


    ###retruns

    z : result of convolutional neural network operation. 
        the input size and the filter size is make new smaple size with stride.
        so we call it 'hopes'. then output size is
        [BATCH, H HOPES, W HOPES, OUT CHANNEL]
    '''

    (B,H,W,C) = x.shape
    (in_C,f_H,f_W,out_C) = ftr.shape
    (strd_H,strd_W) = strd

    H_hopes = int((H - f_H)/strd_H + 1)
    W_hopes = int((W - f_W)/strd_W + 1)

    z = np.zeros((B,H_hopes,W_hopes,out_C))

    for b in range(B):#배치에서 한장을 가져옴
        for out_ch in range(out_C):
            for h in range(H_hopes):#우선 가로로 움직이기 위해 행을 선택해줌
                h_start = h*strd_H
                for w in range(W_hopes):#이제 선택된 행 안에서 움직임
                    w_start = w*strd_W
                    f_sum = 0
                    for ch in range(C):
                        x_ = x[b,
                               h_start:h_start+f_H,
                               w_start:w_start+f_W,
                               ch]
                        f_sum += np.sum(x_*ftr[ch,:,:,out_ch])
                    z[b,h,w,out_ch]=f_sum
    return z

In [3]:
def convolution_transpose(x,
                          ftr,
                          strd):

    '''

    ###args

    x : input of convolution.
            [BATCH, HEIGHT, WIDTH, CHANNEL]
    ftr : filter for convolution operation.
            [IN CHANNEL, FILTER H, FILTER W, OUT CHANNEL]
    strd : filter stride number. 
            [H STRIDE, W STRIDE]


    ###retruns

    z : result of convolutional neural network operation. 
        the input size and the filter size is make new smaple size with stride.
        so we call it 'hopes'. then output size is
        [BATCH, H HOPES, W HOPES, OUT CHANNEL]
    '''

    (B,H,W,C) = x.shape
    (in_C,f_H,f_W,out_C) = ftr.shape
    (strd_H,strd_W) = strd
    
    H_hopes = (H-1)*strd_H+f_H
    W_hopes = (W-1)*strd_W+f_W
    
    z = np.zeros((B,H_hopes,W_hopes,out_C))
    
    for b in range(B):#배치에서 한장을 가져옴
        for out_ch in range(out_C):
            for h in range(H):#우선 가로로 움직이기 위해 행을 선택해줌
                for w in range(W):#이제 선택된 행 안에서 움직임
                    f_sum = 0
                    for ch in range(C):
                        x_ = x[b,h,w,ch]
                        f_sum += x_*ftr[ch,:,:,out_ch]
                    z[b,
                      h*strd_H:(h*strd_H)+f_H,
                      w*strd_W:(w*strd_W)+f_W,
                      out_ch] += f_sum
    return z

In [ ]:
def conv_back_prop(fore_delta,
                   fore_w,
                   fore_strd,
                   d_act,
                   x,
                   fltrs,
                   strd):
    '''
    
    !!!warning!!!
    you need convolution_transpose function in same class.
    
    ###args
    
    fore_delta : a tensor of delta from (l+1)th layer
                when your local layer is (l)th layer.
                it must be same shape as output.
                [BATCH, DELTA_H, DELTA_W, OUT_CANNEL]
                
    fore_w : a convolution filter from (l+1)th layer.
                [IN_CHANNEL, FILTER_H, FILTER_W, OUT_CHANNEL]
    
    fore_strd : a tuple or list of stride in (l+1)th layer.
                [H STRIDE, WSTRIDE]
    
    d_act : a ndarray which is derivative of activation function 
    
    x : input of local layer
        [BATCH, HEIGHT, WIDTH, IN_CHANNEL]
        
    fltrs : a convolution filter tensor of local layer.
                it cna be different shape as fore_w
                [IN_CHANNEL, FILTER_H, FILTER_W, OUT_CHANNEL]
    
    strd : a tuple or list of stride in local layer.
                [H STRIDE, WSTRIDE]
    
    ###returns
    
    z : a partial drivative of convolution filter.
        you need this for update local convolution filter.
        so it must be same shape as local convolution filter.
        [IN CHANNEL, FILTER_H, FILTER_W, OUT_CHANNEL]
    
    '''
    error =  convolution(fore_delta,fore_w,fore_strd)
    delta = error*d_act
    
    SH,SW = strd[0],strd[1]
    (INCH,FH,FW,OUTCH) = filters.shape
    (B,DH,DW,OUTCH) = delta.shape
    
    z = np.zeros_like(fltrs)
    for b in range(B):
        for h in range(DH):
            for w in range(DW):
                for outch in range(OUTCH):
                    for inch in range(INCH):
                        I = inputs[b,
                                   h*SH:h*SH+FH,
                                   w*SW:w*SW+FW,
                                   inch]
                        
                        D = delta[b,h,w,outch]
                        z[inch,:,:,outch] += I*D
    return z

In [ ]:
def conv_transpose_back_prop(fore_delta,
                             fore_w,
                             fore_strd,
                             d_act,
                             x,
                             fltrs,
                             strd):
    '''
    
    !!!warning!!!
    you need convolution function in same class.
    
    ###args
    
    fore_delta : a ndarray of delta from (l+1)th layer
                when your local layer is (l)th layer.
                it must be same shape as output.
                [BATCH, DELTA_H, DELTA_W, OUT_CANNEL]
                
    fore_w : a ndarray of convolution filter from (l+1)th layer.
                [IN_CHANNEL, FILTER_H, FILTER_W, OUT_CHANNEL]
    
    fore_strd : a tuple or list of stride in (l+1)th layer.
                [H STRIDE, W STRIDE]
    
    d_act : a ndarray which is derivative of activation function 
    
    x : input of local layer
        [BATCH, HEIGHT, WIDTH, IN_CHANNEL]
        
    fltrs : a convolution filter tensor of local layer.
                it cna be different shape as fore_w
                [IN_CHANNEL, FILTER_H, FILTER_W, OUT_CHANNEL]
    
    strd : a tuple or list of stride in local layer.
                [H STRIDE, WSTRIDE]
    
    ###returns
    
    z : a partial drivative of convolution_transpose filter.
        you need this for update local convolution_transpose filter.
        so it must be same shape as local convolution_transpose filter.
        [IN CHANNEL, FILTER_H, FILTER_W, OUT_CHANNEL]
    
    '''
    
    error = convolution(fore_delta,fore_w,fore_strd)
    delta = error * d_act
    
    for b in range(B):
        for outch in range(outCH):
            for inch in range(inCH):
                for inh in range(inH):
                    for inw in range(inW):

                        I = x[b,inh,inw,inch]
                        
                        D = delta[b,
                                 inh*sH : inh*sH+fH,
                                 inw*sW : inw*sW+fW,
                                 outch]
                        z[inch,:,:,outch] += I*D
    return z